In [1]:
import pandas as pd
import numpy as np
import torch
import skorch
from skorch import NeuralNetBinaryClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, plot_roc_curve
from skorch.callbacks import EarlyStopping
import time
from skorch.helper import DataFrameTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
price = pd.read_csv("aapl.csv")
price['Close/Last'] = price['Close/Last'].map(lambda x: str(x)[1:])
price['Open'] = price['Open'].map(lambda x: str(x)[1:])
price['High'] = price['High'].map(lambda x: str(x)[1:])
price['Low'] = price['Low'].map(lambda x: str(x)[1:])
price.to_csv("aapl2.csv")
price

,Date,Close/Last,Volume,Open,High,Low
0,05/11/2021,125.91,126142800,123.5,126.27,122.77
1,05/10/2021,126.85,88071230,129.41,129.54,126.81
2,05/07/2021,130.21,78973270,130.85,131.2582,129.475
3,05/06/2021,129.74,78128330,127.89,129.75,127.13
4,05/05/2021,128.1,84000900,129.2,130.45,127.97
...,...,...,...,...,...,...
247,05/18/2020,78.74,135372520,78.2925,79.125,77.581
248,05/15/2020,76.9275,166348360,75.0875,76.975,75.0525
249,05/14/2020,77.385,158929080,76.1275,77.4475,75.3825
250,05/13/2020,76.9125,200622560,78.0375,78.9875,75.8025


In [25]:
pricelist = ["aal.csv","aapl.csv","adi.csv","cern.csv","cost.csv","dish.csv","ebay.csv","fisv.csv","intc.csv","jnj.csv","mnst.csv","msft.csv","nflx.csv","qcom.csv","rtx.csv","siri.csv","tsla.csv","vrtx.csv","wdc.csv"]
namelist = [x[:-4] for x in pricelist]
# aal = pd.read_csv("aal.csv")
# for x in range(len(pricelist)):
#     print(namelist[x])
#     print(pricelist[x])
#     name = namelist[x]
#     y = pd.read_csv(str(pricelist[x]))
    

In [26]:
aal = pd.read_csv("aal.csv")
aapl = pd.read_csv("aapl.csv")
adi = pd.read_csv("adi.csv")
cern = pd.read_csv("cern.csv")
cost = pd.read_csv("cost.csv")
dish = pd.read_csv("dish.csv")
ebay = pd.read_csv("ebay.csv")
fisv = pd.read_csv("fisv.csv")
intc = pd.read_csv("intc.csv")
jnj = pd.read_csv("jnj.csv")
mnst = pd.read_csv("mnst.csv")
msft = pd.read_csv("msft.csv")
nflx = pd.read_csv("nflx.csv")
qcom = pd.read_csv("qcom.csv")
rtx = pd.read_csv("rtx.csv")
siri = pd.read_csv("siri.csv")
tsla = pd.read_csv("tsla.csv")
vrtx = pd.read_csv("vrtx.csv")
wdc = pd.read_csv("wdc.csv")


In [32]:
aal["Symbol"] = "aal"
aapl["Symbol"] = "aapl"
adi["Symbol"] = "adi"
cern["Symbol"] = "cern"
cost["Symbol"] = "cost"
dish["Symbol"] = "dish"
ebay["Symbol"] = "ebay"
fisv["Symbol"] = "fisv"
intc["Symbol"] = "intc"
jnj["Symbol"] = "jnj"
mnst["Symbol"] = "mnst"
msft["Symbol"] = "msft"
nflx["Symbol"] = "nflx"
qcom["Symbol"] = "qcom"
rtx["Symbol"] = "rtx"
siri["Symbol"] = "siri"
tsla["Symbol"] = "tsla"
vrtx["Symbol"] = "vrtx"
wdc["Symbol"] = "wdc"

In [30]:
aal

,Date,Close/Last,Volume,Open,High,Low,Symbol
0,05/11/2021,$21.57,37417770,$21.07,$21.7,$20.91,aal
1,05/10/2021,$22,32570030,$22.23,$22.54,$21.96,aal
2,05/07/2021,$22,32022920,$21.4,$22.19,$21.27,aal
3,05/06/2021,$21.49,25632580,$21.51,$21.75,$21.16,aal
4,05/05/2021,$21.57,27984850,$21.35,$21.87,$21.19,aal
...,...,...,...,...,...,...,...
247,05/18/2020,$9.87,76624410,$9.62,$9.94,$9.36,aal
248,05/15/2020,$9.04,39560530,$8.85,$9.36,$8.76,aal
249,05/14/2020,$9.15,72320520,$8.77,$9.19,$8.25,aal
250,05/13/2020,$9.11,68704640,$9.52,$9.53,$8.83,aal


In [35]:
namelist2 = [aal, aapl, adi, cern, cost, dish, ebay, fisv, intc, jnj, mnst, msft, nflx, qcom, rtx, siri, tsla, vrtx, wdc]
concatdf = pd.concat(namelist2)

In [37]:
concatdf.to_csv('concatdf.csv', index=False)

In [38]:
concatdf2 = pd.read_csv('concatdf2.csv')

In [39]:
concatdf2

,Date,Close/Last,Volume,Open,High,Low,Symbol,Change
0,11/05/2021,21.57,37417770,21.07,21.70,20.9100,aal,0.50
1,10/05/2021,22.00,32570030,22.23,22.54,21.9600,aal,-0.23
2,07/05/2021,22.00,32022920,21.40,22.19,21.2700,aal,0.60
3,06/05/2021,21.49,25632580,21.51,21.75,21.1600,aal,-0.02
4,05/05/2021,21.57,27984850,21.35,21.87,21.1900,aal,0.22
...,...,...,...,...,...,...,...,...
4783,18/05/2020,42.38,6081976,40.81,42.52,40.7000,wdc,1.57
4784,15/05/2020,39.21,5521652,39.04,39.81,38.6050,wdc,0.17
4785,14/05/2020,40.34,5743852,38.61,40.40,37.6500,wdc,1.73
4786,13/05/2020,39.25,6359138,40.80,40.97,38.8400,wdc,-1.55


In [4]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results

In [5]:
premium_search_args = load_credentials("./twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)
premium_search_args


{'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAD4xPgEAAAAACxmJMPiX0adViMH6%2FfBryjgk6SI%3DpOs2TCRAB3eYFsdVq3CfZII8DulDcLnl4fKeTWJdTrCjHaTcNt',
 'endpoint': 'https://api.twitter.com/2/tweets/search/all',
 'extra_headers_dict': None}

In [6]:
rule = gen_rule_payload("%23AAPL%20OR%20%23apple", from_date="2020-05-11", to_date="2021-05-11") # testing with a sandbox account
print(rule)



{"query": "%23AAPL%20OR%20%23apple", "toDate": "202105110000", "fromDate": "202005110000"}


In [7]:
tweets = collect_results(rule,
                         max_results=100,
                         result_stream_args=premium_search_args) # change this if you need to

HTTP Error code: 405: {"title":"Method Not Allowed","type":"about:blank","status":405,"detail":"Method Not Allowed"}
Request payload: {'query': '%23AAPL%20OR%20%23apple', 'toDate': '202105110000', 'fromDate': '202005110000'}


HTTPError: 

In [4]:
import requests
import os
import json
from datetime import datetime
import sys
import time as pytime
from time import sleep
if sys.version_info[0] >= 3:
    from datetime import timezone
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAD4xPgEAAAAACxmJMPiX0adViMH6%2FfBryjgk6SI%3DpOs2TCRAB3eYFsdVq3CfZII8DulDcLnl4fKeTWJdTrCjHaTcNt'

search_url = "https://api.twitter.com/2/tweets/search/all"
next_token = None
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {"query": "#AAPL OR #apple", "end_time": "2021-05-11T13:00:00.00Z", "start_time": "2020-05-11T13:00:00.00Z", "max_results": "500"}


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def pause_until(time):
    """ Pause your program until a specific end time. 'time' is either
    a valid datetime object or unix timestamp in seconds (i.e. seconds
    since Unix epoch) """
    end = time

    # Convert datetime to unix timestamp and adjust for locality
    if isinstance(time, datetime):
        # If we're on Python 3 and the user specified a timezone,
        # convert to UTC and get the timestamp.
        if sys.version_info[0] >= 3 and time.tzinfo:
            end = time.astimezone(timezone.utc).timestamp()
        else:
            zoneDiff = pytime.time() - (datetime.now() - datetime(1970, 1, 1)).total_seconds()
            end = (time - datetime(1970, 1, 1)).total_seconds() + zoneDiff

    # Type check
    if not isinstance(end, (int, float)):
        raise Exception('The time parameter is not a number or datetime object')

    # Now we wait
    while True:
        now = pytime.time()
        diff = end - now

        #
        # Time is up!
        #
        if diff <= 0:
            break
        else:
            # 'logarithmic' sleeping to minimize loop iterations
            sleep(diff / 2)


def connect_to_endpoint(url, headers, next_token):
    if (next_token is not None):
        params = {"query": "#AAPL OR #apple", "end_time": "2021-05-11T13:00:00.00Z", "start_time": "2020-05-11T13:00:00.00Z", "max_results": "500", "tweet.fields": "created_at", "next_token": next_token}
    else:
        params = {"query": "#AAPL OR #apple", "end_time": "2021-05-11T13:00:00.00Z", "start_time": "2020-05-11T13:00:00.00Z", "max_results": "500", "tweet.fields": "created_at"}
    response = requests.request("GET", search_url, headers=headers, params=params)
    
    # https://stackoverflow.com/questions/65109472/how-do-i-loop-my-python-code-for-twitter-api-v2-recent-search
    # Twitter returns (in the header of the request object) how many
    # requests you have left. Lets use this to our advantage
    remaining_requests = int(response.headers["x-rate-limit-remaining"])
    
    # If that number is one, we get the reset-time
    #   and wait until then, plus 15 seconds (you're welcome Twitter).
    # The regular 429 exception is caught below as well,
    #   however, we want to program defensively, where possible.
    if remaining_requests == 1:
        buffer_wait_time = 15
        resume_time = datetime.fromtimestamp( int(response.headers["x-rate-limit-reset"]) + buffer_wait_time )
        print(f"Waiting on Twitter.\n\tResume Time: {resume_time}")
        pause_until(resume_time)  ## Link to this code in above answer

    # We still may get some weird errors from Twitter.
    # We only care about the time dependent errors (i.e. errors
    #   that Twitter wants us to wait for).
    # Most of these errors can be solved simply by waiting
    #   a little while and pinging Twitter again - so that's what we do.
    if response.status_code != 200:

        # Too many requests error
        if response.status_code == 429:
            buffer_wait_time = 60
            resume_time = datetime.fromtimestamp( int(response.headers["x-rate-limit-reset"]) + buffer_wait_time )
            print(f"Waiting on Twitter.\n\tResume Time: {resume_time}")
            pause_until(resume_time)  ## Link to this code in above answer
            print("hi, pause for too many requests over now")

        # Twitter internal server error
        elif response.status_code == 500:
            # Twitter needs a break, so we wait 30 seconds
            resume_time = datetime.now().timestamp() + 30
            print(f"Waiting on Twitter.\n\tResume Time: {resume_time}")
            pause_until(resume_time)  ## Link to this code in above answer

        # Twitter service unavailable error
        elif response.status_code == 503:
            # Twitter needs a break, so we wait 30 seconds
            resume_time = datetime.now().timestamp() + 30
            print(f"Waiting on Twitter.\n\tResume Time: {resume_time}")
            pause_until(resume_time)  ## Link to this code in above answer

        # If we get this far, we've done something wrong and should exit
#         raise Exception(
#             "Request returned an error: {} {}".format(
#                 response.status_code, response.text
#             )
#         )

    # Each time we get a 200 response, lets exit the function and return the response.json
    if response.ok:
        return response.json()
#     print(response.status_code)
#     if response.status_code != 200:
#         raise Exception(response.status_code, response.text)
#     return response.json()

count = 0
flag = True

# def main():
headers = create_headers(bearer_token)
finalset = []
while flag:
    # Replace the count below with the number of Tweets you want to stop at. 
    # Note: running without the count check will result in getting more Tweets
    # that will count towards the Tweet cap
    if count >= 5000000:
        break
    q = "#AAPL OR #apple"
    sleep(2)
    json_response = connect_to_endpoint(search_url, headers, next_token)
    result_count = json_response['meta']['result_count']
    if 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']
        if result_count is not None and result_count > 0 and next_token is not None:
#             for tweet in json_response['data']:
#                 print(tweet['id'])
            count += result_count
            print(count)
            print(next_token)
            json_response = connect_to_endpoint(search_url, headers, next_token)
        if json_response is not None:
            finalset.append(json_response['data'])
    else:
        flag = False
print("Total Tweet IDs saved: {}".format(count))
    
#     json_response = connect_to_endpoint(search_url, headers)
# print(json.dumps(json_response, indent=4, sort_keys=True))


# if __name__ == "__main__":
#     main()

496
b26v89c19zqg8o3foswrect7diwhw2oy4ckkf4a7bmqgt
Waiting on Twitter.
	Resume Time: 2021-06-12 13:38:39
hi, pause for too many requests over now
983
b26v89c19zqg8o3foswrecslpnrhyj55udf7c0eouie7x
1469
b26v89c19zqg8o3foswrecrpmyxcdvgq0t9pbqc8hbfjx
1960
b26v89c19zqg8o3foswrecr3rjqnu2rfpvbcuoalewrr1
2454
b26v89c19zqg8o3foswqzqdssi1loc105yzfakeau8j25
2942
b26v89c19zqg8o3foswqzqcwf6mwstxrxiobh2dxtn7ul
3432
b26v89c19zqg8o3foswqzqc00eynt9j1oh5p9eo4nfou5
3920
b26v89c19zqg8o3foswqzo9dckfhaie9u4m8f8fq0gakd
4398
b26v89c19zqg8o3foswqzo8h3t3ocz7yjur3yu8wg7xbx
4886
b26v89c19zqg8o3foswqzo7vkheehl7v2ce86q9n54lfh
Waiting on Twitter.
	Resume Time: 2021-06-12 13:54:41
hi, pause for too many requests over now
5362
b26v89c19zqg8o3foswqzo7a147kgrffiyjjua5bbiuf1
5836
b26v89c19zqg8o3foswqzo6oerhav8hc7zlvtsoygi1z1
6329
b26v89c19zqg8o3foswqzm4cnvbvvpmrxf1si2lqhuzjx
Waiting on Twitter.
	Resume Time: 2021-06-12 14:10:43
hi, pause for too many requests over now
6818
b26v89c19zqg8o3foswqzm3r621jvxgbdlw41yqsd6z5p
730

KeyboardInterrupt: 

In [9]:
print(finalset[0][0])
print(finalset[50][0])

{'created_at': '2021-05-11T10:18:35.000Z', 'id': '1392061565185236994', 'text': 'RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…'}
{'created_at': '2021-05-07T00:32:55.000Z', 'id': '1390464625901146112', 'text': 'おはようございます！染谷です！\n金曜日な感じが全くしません🤣が、本日も一日宜しくお願いします✨\n\n※GW中にご購入いただいたお客様、本日には全てお届けの手配が出来る予定です！もう少々お待ちくださいませ！\n\n#Mac #Apple #楽天 #Yahoo #ヤフオク #ポイ活 #中古 #リユース #おうち時間充実'}


In [7]:
print(next_token)

b26v89c19zqg8o3fosns2v1e9xg59w0fbzcscj070467x


In [5]:
print(len(finalset))

1071


In [6]:
with open('twitter_aapl_1.json', 'w') as file:
    json.dump(finalset , file)

In [72]:
print(finalset[0])

[{'created_at': '2021-05-11T11:44:22.000Z', 'id': '1392083153380995072', 'text': 'RT @DomainFQ: Listed For Sale \n\nhttps://t.co/qOjgV5IuAm \n\n#fashion #music #students #apple #100DaysOfCode #coach #university #english  #mar…'}, {'created_at': '2021-05-11T11:44:22.000Z', 'id': '1392083153267875840', 'text': 'RT @DomainFQ: .\nDomain For Sale \n\nhttps://t.co/qOjgV5IuAm \n\n#fashion #music #apple #100DaysOfCode #coach #university #english #martialarts…'}, {'created_at': '2021-05-11T11:44:19.000Z', 'id': '1392083144556171265', 'text': 'RT @DomainFQ: .\nDomain For Sale \n\nhttps://t.co/qOjgV5IuAm \n\n#fashion #music #apple #100DaysOfCode #coach #university #english #martialarts…'}, {'created_at': '2021-05-11T11:44:19.000Z', 'id': '1392083141494444035', 'text': 'RT @DomainFQ: .\nDomain Name For Sale\n\nhttps://t.co/27xdV5ITgb\n\n#ElectricWorld #Ev #eBike #javascript #python #ecar #autos #ElectricVehicle…'}, {'created_at': '2021-05-11T11:44:14.000Z', 'id': '1392083119663104001', 'text': 'RT

In [66]:
print(finalset[0][0].keys())

dict_keys(['created_at', 'id', 'text'])


In [62]:
import requests


def connect_to_endpoint(bearer_token, query, next_token=None):
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    # add additional parameters as needed
    params = {"tweet.fields" : "created_at"}
    qu
    # replace appropriate start and end times below
    if (next_token is not None):
        url = "https://api.twitter.com/2/tweets/search/all?max_results=500&query={}&start_time=2006-03-31T15:00:00Z&next_token={}".format(query, next_token)
    else:
        url = "https://api.twitter.com/2/tweets/search/all?max_results=500&start_time=2006-03-31T15:00:00Z&query={}".format(query)
    
    response = requests.request("GET", url, params=params, headers=headers)
    
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    
    return response.json()


count = 0
flag = True

# Replace with your own bearer token from your academic project in developer portal
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAD4xPgEAAAAACxmJMPiX0adViMH6%2FfBryjgk6SI%3DpOs2TCRAB3eYFsdVq3CfZII8DulDcLnl4fKeTWJdTrCjHaTcNt'

while flag:
    # Replace the count below with the number of Tweets you want to stop at. 
    # Note: running without the count check will result in getting more Tweets
    # that will count towards the Tweet cap
    if count >= 1000:
        break
    q = "#AAPL OR #apple"
    json_response = connect_to_endpoint(bearer_token, q)
    result_count = json_response['meta']['result_count']
    if 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']
        if result_count is not None and result_count > 0 and next_token is not None:
#             for tweet in json_response['data']:
#                 print(tweet['id'])
            count += result_count
            print(count)
            json_response = connect_to_endpoint(bearer_token, q, next_token)
    else:
        flag = False
print("Total Tweet IDs saved: {}".format(count))

NameError: name 'qu' is not defined